In [ ]:
# CNN을 이용하여 개, 고양이 이미지 분류(이항분류)
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt
import keras

In [ ]:
data_url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
pathto_zip = keras.utils.get_file('cats_and_dogs.zip', origin=data_url, extract=True)  # 내가 줄 파일이름
PATH = os.path.join(os.path.dirname(pathto_zip), 'cats_and_dogs_filtered')  # 원본파일 이름
print(PATH)

batch_size = 128
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150

# 데이터 준비
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
print(train_cats_dir)

num_cats_dir = len(os.listdir(train_cats_dir))
num_dogs_dir = len(os.listdir(train_dogs_dir))
print(os.listdir(train_cats_dir))
num_cats_val = len(os.listdir(validation_cats_dir))
num_dogs_val = len(os.listdir(validation_dogs_dir))

total_train = num_cats_dir + num_dogs_dir
total_val = num_cats_val + num_dogs_val

print('total_train cat : ', num_cats_dir)  # 1000
print('total_train dog : ', num_dogs_dir)  # 1000
print('total_validation dog : ', num_cats_val)  # 500
print('total_validation dog : ', num_dogs_val)  # 500
print('total train : ', total_train)  # 2000
print('total validation : ', total_val)  # 1000

In [ ]:
# 데이터를 실수 타입의 텐서로 전처리
# 2개의 이미지에 대한 라벨링
train_image_gen = ImageDataGenerator(rescale = 1./255)
validation_image_gen = ImageDataGenerator(rescale = 1./255)

train_data_gen = train_image_gen.flow_from_directory(directory=train_dir, batch_size=batch_size, shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH), class_mode='binary')  # (128, 150, 150, 3)  128개씩 150바이150 채널3
val_data_gen = validation_image_gen.flow_from_directory(directory=validation_dir, batch_size=batch_size,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH), class_mode='binary')
# flow_from_directory는 배치를 부한정 만들어 내므로 break를 해주는 것이 바람직
for a, b, in train_data_gen:
  print(a.shape, ' ', b.shape)  # (128, 150, 150, 3)   (128,) 트레인데이터, 라벨링
  print(b[0], b[1])
  break

for a, b, in val_data_gen:
  break

# 데이터 확인
sample_training_images, _ = next(train_data_gen)

def plotImage(img_arr):
  fig, axes = plt.subplots(1, 5, figsize=(20,20))
  axes = axes.flatten()
  for img, ax in zip(img_arr, axes):
    ax.axis('off')  # 축은 볼 필요 없음
    ax.imshow(img)
  plt.tight_layout()
  plt.show()

plotImage(sample_training_images[:5])
# print(sample_training_images[:1])  ...[0.18823531 0.2901961  0.24705884]]]]

In [ ]:
# 모델
model = Sequential([
    Conv2D(filters=16, kernel_size=(3, 3), strides=(1,1), padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(rate=0.2),
    Conv2D(filters=32, kernel_size=(3, 3), strides=(1,1), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(rate=0.2),
    Conv2D(filters=64, kernel_size=(3, 3), strides=(1,1), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(rate=0.2),
    Flatten(),
    Dense(units=128, activation='relu'),
    Dense(units=64, activation='relu'),
    Dense(units=1),
])

model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])  # 수치적으로 안정적
print(model.summary())

In [ ]:
# 학습
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch = total_train//batch_size,
    epochs=epochs,
    validation_data = val_data_gen,
    validation_steps = total_val // batch_size
)

model.save('cat_dog.keras')

In [ ]:
# 학습 결과 시각화
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1,2,1)
plt.plot(epochs_range, acc, label='accuracy')
plt.plot(epochs_range, val_acc, label='val_accuracy')
plt.legend(loc='best')
plt.subplot(1,2,2)
plt.plot(epochs_range, loss, label='loss')
plt.plot(epochs_range, val_loss, label='val_loss')
plt.legend(loc='best')
plt.show()

In [ ]:
#새로운 이미지를 분류
import numpy as np
from google.colab import files
import tensorflow as tf

mymodel=tf.keras.models.load_model('cat_dog.keras')
uploaded=files.upload()
print(uploaded.keys())

for fn in uploaded.keys():
    path='/content/'+ fn
    img = tf.keras.utils.load_img(path, target_size=(150,150))
    x=tf.keras.utils.img_to_array(img)
    print(x.shape)

    x=np.expand_dims(x,axis=0)
    print(x.shape)
    images=np.vstack([x])
    classes=mymodel.predict(images,batch_size=10)
    print(classes)
    print(classes[0])

    if classes[0] > 0:
        print(fn+'은 댕댕이야')
    else:
        print(fn+'은 양이')